# **순전파**

In [1]:
import numpy as np
# ReLU 함수 정의
def ReLU(X):
    return np.maximum(0,X)
# 입력층 데이터(1*3)
x = np.array([[0.5, 0.8, 0.2]])
# 은닉층 가중치(3*2), 편향(1*2)
W1 = np.array([[1.0, -1.0],
               [0.0, 2.0],
               [1.0, 0.5]])
b1 = np.array([[0.1, 0.2]])
# 은닉층 계산
z1 = x @ W1 + b1
a1 = ReLU(z1)
# 출력층 가중치(2*1), 편향(1*1)
W2 = np.array([[2.0],
               [-1.0]])
b2 = np.array([[0.5]])
# 출력층 계산
z2 = a1 @ W2 + b2
y_pred = ReLU(z2)
# 결과 출력
print("입력 벡터 x:", x)
print("은닉층 선형 출력 z1:", z1)
print("은닉층 활성화 a1 (ReLU):", a1)
print("출력층 선형 출력 z2:", z2)
print("최종 예측 y_pred (ReLU):", y_pred)

입력 벡터 x: [[0.5 0.8 0.2]]
은닉층 선형 출력 z1: [[0.8 1.4]]
은닉층 활성화 a1 (ReLU): [[0.8 1.4]]
출력층 선형 출력 z2: [[0.7]]
최종 예측 y_pred (ReLU): [[0.7]]


# **역전파 (직접 미분)**

In [ ]:
import numpy as np

# ReLU 함수와 도함수 정의
def ReLU(X):
    return np.maximum(0, X)

def relu_deriv(X):
    return (x > 0).astype(float)
# 입력층 데이터(1*3)
x = np.array([[0.5, 0.8, 0.2]])
# 은닉층 가중치(3*2), 편향(1*2)
W1 = np.array([[1.0, -1.0],
               [0.0, 2.0],
               [1.0, 0.5]])
b1 = np.array([[0.1, 0.2]])
# 은닉층 계산
z1 = x @ W1 + b1
a1 = ReLU(z1)
# 출력층 가중치(2*1), 편향(1*1)
W2 = np.array([[2.0],
               [-1.0]])
b2 = np.array([[0.5]])
# 순전파 계산
z2 = a1 @ W2 + b2
y_pred = ReLU(z2)
# 손실함수
loss = 0.5 * (y_pred - y) ** 2

# --- 역전파 ---

# 출력층 계산
# dL_dW2 = dL_dy * dy_dz2 * dz2_dW2
dL_dy = y_pred - y                     # dL/dy_pred
dy_dz2 = relu_deriv(z2)                # ReLU 미분
dz2 = dL_dy * dy_dz2                   # dL/dz2 (1,1)

dW2 = a1.T @ dz2                       # (2,1)
db2 = dz2                              # (1,1)

# 은닉층 계산
dz1 = dz2 @ W2.T * relu_deriv(z1)      # (1,2)
dW1 = x.T @ dz1                        # (3,2)
db1 = dz1                              # (1,2)

# 가중치 및 편향 업데이트
lr = 0.1

W2 -= lr * dW2
b2 -= lr * db2
W1 -= lr * dW1
b1 -= lr * db1

# 두 번째 순전파
z1_2 = x @ W1 + b1
a1_2 = ReLU(z1_2)
z2_2 = a1_2 @ W2 + b2
y_pred_2 = ReLU(z2_2)
loss_2 = 0.5 * (y_pred_2 - y) ** 2
# 결과 출력
print("---순전파---")
print("z1:", z1)
print("a1:", a1)
print("z2:", z2)
print("y_pred:", y_pred)
print("loss:", loss.item())

print("\n---역전파---")
print("dW2:", dW2)
print("db2:", db2)
print("dW1:", dW1)
print("db1:", db1)

print("\n---업데이트 후---")
print("W1:", W1)
print("b1:", b1)
print("W2:", W2)
print("b2:", b2)

print("\n---두 번째 순전파---")
print("z1:", z1_2)
print("a1:", a1_2)
print("z2:", z2_2)
print("y_pred:", y_pred_2)
print("loss:", loss_2.item())

# **역전파(pytorch 사용)**

In [9]:
import torch
import torch.nn.functional as F
# 입력 데이터와 정답
x = torch.tensor([[0.5, 0.8, 0.2]], dtype = torch.float32)
y = torch.tensor([[1.0]], dtype = torch.float32)

# 가중치 및 편향
W1 = torch.tensor([[1.0, -1.0],
                   [0.0, 2.0],
                    [1.0, 0.5]], requires_grad = True)
b1 = torch.tensor([[0.1, 0.2]], dtype = torch.float32, requires_grad = True)

W2 = torch.tensor([[2.0],
                   [-1.0]], dtype=torch.float32, requires_grad=True)

b2 = torch.tensor([[0.5]], dtype=torch.float32, requires_grad=True)

# 순전파
z = x @ W1 + b1
a = F.relu(z)
z2 = a @ W2 + b2
y_pred = F.relu(z2)
loss = 0.5 * (y_pred -y) ** 2
# 역전파
loss.backward()
# 파라미터 업데이트
lr = 0.1
with torch.no_grad():
    W1 -= lr * W1.grad
    b1 -= lr * b1.grad
    W2 -= lr * W2.grad
    b2 -= lr * b2.grad
# 두 번째 순전파
z1_2 = x @ W1 + b1
a1_2 = torch.relu(z1_2)
z2_2 = a1_2 @ W2 + b2
y_pred_2 = torch.relu(z2_2)
loss_2 = 0.5 * (y_pred_2 - y) ** 2
# 결과 출력
print("y_pred : ", y_pred.item())
print("loss : ", loss.item())
print("y_pred_2 : ", y_pred_2.item())
print("loss_2 : ", loss_2.item())

y_pred :  0.6999999284744263
loss :  0.045000020414590836
y_pred_2 :  1.09784734249115
loss_2 :  0.004787051118910313


# **순전파+역전파 for문**

In [11]:
import torch
import torch.nn.functional as F
# 입력 데이터와 정답
x = torch.tensor([[0.5, 0.8, 0.2]], dtype = torch.float32)
y = torch.tensor([[1.0]], dtype = torch.float32)

# 가중치 및 편향
W1 = torch.tensor([[1.0, -1.0],
                   [0.0, 2.0],
                    [1.0, 0.5]], requires_grad = True)
b1 = torch.tensor([[0.1, 0.2]], dtype = torch.float32, requires_grad = True)

W2 = torch.tensor([[2.0],
                   [-1.0]], dtype=torch.float32, requires_grad=True)

b2 = torch.tensor([[0.5]], dtype=torch.float32, requires_grad=True)

# 학습 설정
lr = 0.1
epochs = 10

print("---학습 시작---")
for epoch in range(1, epochs+1):
  #순전파
  z = x @ W1 + b1
  a = F.relu(z)
  z2 = a @ W2 + b2
  y_pred = F.relu(z2)
  loss = 0.5 * (y_pred - y) ** 2

  #역전파
  loss.backward()

  # 파라미터 업데이트
  lr = 0.1
  with torch.no_grad():
      W1 -= lr * W1.grad
      b1 -= lr * b1.grad
      W2 -= lr * W2.grad
      b2 -= lr * b2.grad

  #기울기 초기화
  W1.grad.zero_()
  b1.grad.zero_()
  W2.grad.zero_()
  b2.grad.zero_()

  print("Epoch", epoch, "| y_pred: ", y_pred.item(), "| loss: ", loss.item())

---학습 시작---
Epoch 1 | y_pred:  0.6999999284744263 | loss:  0.045000020414590836
Epoch 2 | y_pred:  1.09784734249115 | loss:  0.004787051118910313
Epoch 3 | y_pred:  0.967643141746521 | loss:  0.0005234831478446722
Epoch 4 | y_pred:  1.0106124877929688 | loss:  5.6312448577955365e-05
Epoch 5 | y_pred:  0.9965109825134277 | loss:  6.086621397116687e-06
Epoch 6 | y_pred:  1.0011465549468994 | loss:  6.572940947080497e-07
Epoch 7 | y_pred:  0.9996230602264404 | loss:  7.104179644557007e-08
Epoch 8 | y_pred:  1.0001239776611328 | loss:  7.685230229981244e-09
Epoch 9 | y_pred:  0.9999591112136841 | loss:  8.359464231944003e-10
Epoch 10 | y_pred:  1.0000135898590088 | loss:  9.234213393938262e-11
